<a href="https://colab.research.google.com/github/KhoomeiK/MindMapResearch/blob/transformer/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [0]:
# ! pip install -U -q PyDrive
import torch
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

SEED = 187

torch.manual_seed(SEED)

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded2 = drive.CreateFile({'id': '1HvZH9ys_ufTF-wcRMNFSKS7lT3pJu6YY'}) 
downloaded2.GetContentFile('labels.pkl')

downloaded = drive.CreateFile({'id': '1BqSv5mgxsyP5rhMoAAKFyvPtAH8XLcO6'}) 
downloaded.GetContentFile('embeddings.zip')

! unzip embeddings.zip

In [0]:
import pickle, os
from torch.utils.data import DataLoader, Dataset, random_split


class Users(Dataset):
    def __init__(self, embedPath, labelPath): 
        self.users, self.labels = [], []
        users = {}
        labels = pickle.load(open(labelPath, 'rb'))
        file_list = os.listdir(embedPath)

        for user in file_list:
            users[user[:-4]] = pickle.load(open(
                os.path.join(embedPath, user), 'rb'))

        intersection = list(set(users.keys()).intersection(set(labels.keys())))
        for i in intersection:
            userTensor = torch.tensor([users[i]])
            if list(userTensor.shape)[1] != 1:
                self.users.append(userTensor)
                self.labels.append(torch.tensor([labels[i][0]]))

        print(len(self), self[0])

    def __getitem__(self, i):
        return self.users[i], self.labels[i]
    
    def __len__(self):
        assert len(self.users) == len(self.labels)
        return len(self.users)

train, valid, test = random_split(Users('user_subset', 'labels.pkl'),
                                  [4280, 538, 538])

# Model


In [0]:
import torch
import numpy as np
import torch.nn as nn
import torch.functional as F
from sklearn.decomposition import PCA


class PCALayer(nn.Module):
    def __init__(self, n=256):
        super(PCA, self).__init__()
        self.n = n
        self.pca = PCA(n_components=self.n)

    def forward(self, X):
        self.pca.fit(X.numpy())
        return torch.from_numpy(self.pca.transform(X))


class MapNet(nn.Module):
    def __init__(self, n=4096):
        super(MapNet, self).__init__()
        self.fc_1 = nn.Linear(n, int(n ** 0.5))
        self.fc_2 = nn.Linear(int(n ** 0.5), 4 * int(n ** 1/2))

    def forward(self, X):
        output = self.fc_1(X)
        output = self.fc_2(output)
        return output


class CustomMHAttention(nn.Module):
    def __init__(self, pca_n=False, n_heads=8):
        super(CustomMHAttention, self).__init__()
        self.n_heads = n_heads
        self.dim_red = MapNet() if not pca_n else PCALayer(pca_n)
        self.attn = nn.MultiheadAttention(256, n_heads)
        self.fc = nn.Linear(256, 1)

    def forward(self, X):
        red_output = self.dim_red(X)
        output = nn.Dropout(output)
        output = self.attn(output, output, output)
        output = self.fc(output)
        output = F.sigmoid(output)


# Training

In [0]:
from tensorflow import summary
import datetime
%load_ext tensorboard
from google.colab import files

current_time = str(datetime.datetime.now().timestamp())
train_log_dir = '/logs/tensorboard/train/' + current_time
test_log_dir = '/logs/tensorboard/test/' + current_time
train_summary_writer = summary.create_file_writer(train_log_dir)
test_summary_writer = summary.create_file_writer(test_log_dir)

%tensorboard --logdir logs/tensorboard

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 10
lr = 0.001
globaliter = 0

model = CustomMHAttention().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()
model = model.train()

train_losses, valid_losses = [], []

for epoch in range(epochs):
    print('TRAINING -- EPOCH {}/{}'.format(epoch, epochs))
    train_loss, total = 0, 0
    for X, Y in train:
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        loss = criterion(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        total += 1
        
        with train_summary_writer.as_default():
            tf.summary.scalar('train_loss', loss.item(), step=globaliter)

        if total % 300 == 0:
            print('X: {:.4f}    y: {:.4f}    y_pred:{:.4f}'.format(pred.item(), Y.item(), loss.item()))

    train_losses.append(train_loss / total)
    print('EPOCH {}    Train Loss: {:.4f}'.format(epoch, train_losses[-1]))

    with torch.no_grad():
        print('VALIDATION -- EPOCH {}/{}'.format(epoch, epochs))
        valid_loss, valid_total = 0, 0
        for X, Y in valid:
            X, Y = X.to(device), Y.to(device)
            pred = model(X)
            loss = criterion(pred, Y)

            valid_loss += loss.item()
            valid_total += 1
            
            if total % 100 == 0:
                print('X: {:.4f}    y: {:.4f}    y_pred:{:.4f}'.format(pred.item(), Y.item(), loss.item()))

            with train_summary_writer.as_default():
                tf.summary.scalar('valid_loss', loss.item(), step=globaliter)
        
        valid_losses.append(valid_loss / valid_total)
        print('EPOCH {}    Validation Loss: {:.4f}'.format(epoch, train_losses[-1]))
    
    globaliter += 1

print(train_losses)
print(valid_losses)

!zip -r /content/logs.zip /content/logs/
files.download("/content/logs.zip")

# Results and Visualizations

In [0]:
torch.no_grad()
total = 0
test_losses = []
preds = []

for X, Y in test:
    X, Y = X.to(device), Y.to(device)
    pred = model(X)
    loss = criterion(pred, Y)

    preds.append(pred)
    test_losses.append(loss.item())
    total += 1
    
    if total % 100 == 0:
        print('X: {:.4f}    y: {:.4f}    y_pred:{:.4f}'.format(pred.item(), Y.item(), loss.item()))

test_loss = sum(test_losses) / total
print(test_loss)

In [0]:
labels = [data[1].item() for data in test]

plt.title('Test Data Comparison Plot')
plt.xlabel('label')
plt.ylabel('pred')
plt.scatter(labels, preds, c='#22a6b3')
plt.plot(np.unique(labels), np.poly1d(np.polyfit(labels, preds, 1))(np.unique(labels)), color="#eb4d4b")
plt.plot(np.unique(labels), np.unique(labels), color="#6ab04c")